In [1]:
import tensorflow as tf
import numpy as np
import os, time, random
from utils import str2bool, get_logger, get_entity
from data import random_embedding,tag2label
import datapreprocessing as dp
from model import BiLSTM_CRF

C:\Users\Ian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
## Session configuration
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # default: 0
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2  # need ~700MB GPU memory

In [3]:
#dp.buildCorpus('D:\\Desktop\\产品\\辅诊\\已标\\merged.csv','D:\\Desktop\\产品\\辅诊\\已标\\aa.txt')
word2id = dp.buildVocab('word2id.pkl','D:\\Desktop\\产品\\辅诊\\已标\\aa.txt')

1011


In [5]:
## get char embeddings
word2id = dp.loadVocab('word2id.pkl')
type(word2id)

vocab_size: 1011


dict

In [4]:
word2id.keys()

dict_keys(['有', '糖', '尿', '病', '史', '多', '年', '，', '目', '前', '无', '口', '渴', '食', '等', '症', '状', '。', '今', '药', '服', '完', '配', '患', '者', '三', '周', '明', '显', '诱', '因', '下', '出', '现', '心', '悸', '、', '胸', '闷', '痛', '气', '急', '不', '适', '面', '色', '黧', '黑', '胃', '纳', '可', '二', '便', '调', '夜', '寐', '安', '于', '<NUM>', '月', '始', '颈', '项', '肩', '背', '部', '板', '滞', '酸', '伴', '头', '晕', '近', '余', '日', '加', '重', '入', '后', '尤', '甚', ',', '劳', '累', '健', '康', '证', '体', '检', '时', '咳', '嗽', '痰', '咯', '血', '慢', '性', '支', '管', '炎', '哮', '喘', '关', '节', '外', '提', '咽', '天', '发', '热', '长', '期', '平', '素', '按', '情', '稳', '定', '耳', '鸣', '自', '诉', '压', '控', '制', '否', '认', '物', '过', '敏', '腰', '膝', '疼', '如', '针', '刺', '固', '移', '遇', '寒', '改', '善', '溃', '疡', '治', '疗', '中', '胀', '蒙', '恶', '呕', '吐', '行', '走', '好', '转', '鼻', '塞', '复', '诊', '仍', '流', '涕', '白', '轻', '度', '隐', '嗳', '作', '.', '大', '小', '正', '常', '泻', '一', '般', '受', '凉', '起', '少', '感', '冒', '干', '高', '为', '克', '频', '<UNK>', '<PAD>'])

In [4]:
# if args.pretrain_embedding == 'random':
#     embeddings = random_embedding(word2id, args.embedding_dim)
# else:
#     embedding_path = 'pretrain_embedding.npy'
#     embeddings = np.array(np.load(embedding_path), dtype='float32')
#embeddings = random_embedding(word2id, 300)
embeddings = np.load('pretrain_embedding.npy')
type(embeddings),embeddings.shape

(numpy.ndarray, (1011, 300))

In [5]:
## read corpus and get training data
# if args.mode != 'demo':
#     train_path = os.path.join('.', args.train_data, 'train_data')
#     test_path = os.path.join('.', args.test_data, 'test_data')
#     train_data = read_corpus(train_path)
#     test_data = read_corpus(test_path)
#     test_size = len(test_data)
# train_path = os.path.join('.', 'data_path', 'train_data')
# test_path = os.path.join('.', 'data_path', 'test_data')
train_data = dp.readCorpus('D:\\Desktop\\产品\\辅诊\\已标\\aa.txt')
test_data = dp.readCorpus('D:\\Desktop\\产品\\辅诊\\已标\\aa.txt')
test_size = len(test_data)
len(test_data),test_data[:2]

(520,
 [(['有',
    '糖',
    '尿',
    '病',
    '史',
    '多',
    '年',
    '，',
    '目',
    '前',
    '无',
    '口',
    '渴',
    '，',
    '无',
    '多',
    '尿',
    '，',
    '无',
    '多',
    '食',
    '等',
    '症',
    '状',
    '。',
    '今',
    '药',
    '服',
    '完',
    '配',
    '药',
    '。'],
   ['O',
    'B-B',
    'I-B',
    'I-B',
    'I-B',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B-1ZS',
    'I-1ZS',
    'O',
    'O',
    'B-1ZS',
    'I-1ZS',
    'O',
    'O',
    'B-1ZS',
    'I-1ZS',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']),
  (['患',
    '者',
    '三',
    '周',
    '无',
    '明',
    '显',
    '诱',
    '因',
    '下',
    '出',
    '现',
    '心',
    '悸',
    '、',
    '胸',
    '闷',
    '，',
    '无',
    '胸',
    '痛',
    '、',
    '气',
    '急',
    '等',
    '不',
    '适',
    '，',
    '面',
    '色',
    '黧',
    '黑',
    '，',
    '胃',
    '纳',
    '可',
    '，',
    '二',
    '便',
    '调',
    '，',
    '夜',
    '

In [6]:
## paths setting
paths = {}
# timestamp = str(int(time.time())) if args.mode == 'train' else args.demo_model
timestamp = str(int(time.time()))
output_path = os.path.join('.', 'data_path'+"_save", timestamp)
if not os.path.exists(output_path): os.makedirs(output_path)
summary_path = os.path.join(output_path, "summaries")
paths['summary_path'] = summary_path
if not os.path.exists(summary_path): os.makedirs(summary_path)
model_path = os.path.join(output_path, "checkpoints/")
if not os.path.exists(model_path): os.makedirs(model_path)
ckpt_prefix = os.path.join(model_path, "model")
paths['model_path'] = ckpt_prefix
result_path = os.path.join(output_path, "results")
paths['result_path'] = result_path
if not os.path.exists(result_path): os.makedirs(result_path)
log_path = os.path.join(result_path, "log.txt")
paths['log_path'] = log_path
# get_logger(log_path).info(str(args))
get_logger(log_path).info('BiLSTM-CRF for Chinese NER task')

BiLSTM-CRF for Chinese NER task


In [7]:
## training model
# if args.mode == 'train':
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()

#     ## hyperparameters-tuning, split train/dev
#     # dev_data = train_data[:5000]; dev_size = len(dev_data)
#     # train_data = train_data[5000:]; train_size = len(train_data)
#     # print("train data: {0}\ndev data: {1}".format(train_size, dev_size))
#     # model.train(train=train_data, dev=dev_data)

#     ## train model on the whole training data
#     print("train data: {}".format(len(train_data)))
#     model.train(train=train_data, dev=test_data)  # use test_data as the dev_data to see overfitting phenomena


# model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths=paths, config=config)
model.build_graph()

## hyperparameters-tuning, split train/dev
# dev_data = train_data[:5000]; dev_size = len(dev_data)
# train_data = train_data[5000:]; train_size = len(train_data)
# print("train data: {0}\ndev data: {1}".format(train_size, dev_size))
# model.train(train=train_data, dev=dev_data)

## train model on the whole training data
print("train data: {}".format(len(train_data)))
model.train(train=train_data, dev=test_data)  # use test_data as the dev_data to see overfitting phenomena
# tf.nn.embedding_lookup()

C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


train data: 520


2018-06-14 11:47:28 epoch 1, step 1, loss: 234.1, global_step: 1


2018-06-14 11:47:28 epoch 1, step 9, loss: 92.82, global_step: 9
===========validation / test===========


2018-06-14 11:48:39 epoch 2, step 1, loss: 91.78, global_step: 10


...

2018-06-14 12:32:51 epoch 38, step 9, loss: 5.076, global_step: 342
===========validation / test===========


2018-06-14 12:33:58 epoch 39, step 1, loss: 11.04, global_step: 343


2018-06-14 12:33:58 epoch 39, step 9, loss: 6.123, global_step: 351
===========validation / test===========


2018-06-14 12:35:16 epoch 40, step 1, loss: 8.656, global_step: 352


2018-06-14 12:35:16 epoch 40, step 9, loss: 6.687, global_step: 360
===========validation / test===========


In [8]:
np.save('pretrain_embedding.npy',model.embeddings)

In [12]:
## testing model
# elif args.mode == 'test':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     print("test data: {}".format(test_size))
#     model.test(test_data)
## testing model
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
# model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
# model.build_graph()
print("test data: {}".format(test_size))
model.test(test_data)

=========== testing ===========


.\data_path_save\1526890202\checkpoints/model-31680
test data: 4631
INFO:tensorflow:Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


Restoring parameters from .\data_path_save\1526890202\checkpoints/model-31680


NotFoundError: Key words_2/_word_embeddings not found in checkpoint
	 [[Node: save_1/RestoreV2_28 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2_28/tensor_names, save_1/RestoreV2_28/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_28', defined at:
  File "C:\Users\Ian\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Ian\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\Users\Ian\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    

In [10]:
## demo
# elif args.mode == 'demo':
#     ckpt_file = tf.train.latest_checkpoint(model_path)
#     print(ckpt_file)
#     paths['model_path'] = ckpt_file
#     model = BiLSTM_CRF(args, embeddings, tag2label, word2id, paths, config=config)
#     model.build_graph()
#     saver = tf.train.Saver()
#     with tf.Session(config=config) as sess:
#         print('============= demo =============')
#         saver.restore(sess, ckpt_file)
#         while(1):
#             print('Please input your sentence:')
#             demo_sent = input()
#             if demo_sent == '' or demo_sent.isspace():
#                 print('See you next time!')
#                 break
#             else:
#                 demo_sent = list(demo_sent.strip())
#                 demo_data = [(demo_sent, ['O'] * len(demo_sent))]
#                 tag = model.demo_one(sess, demo_data)
#                 PER, LOC, ORG = get_entity(tag, demo_sent)
#                 print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))
ckpt_file = tf.train.latest_checkpoint(model_path)
print(ckpt_file)
paths['model_path'] = ckpt_file
model = BiLSTM_CRF(embeddings, tag2label, word2id, paths, config=config)
model.build_graph()
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    print('============= demo =============')
    saver.restore(sess, ckpt_file)
    while(1):
        print('Please input your sentence:')
        demo_sent = input()
        if demo_sent == '' or demo_sent.isspace():
            print('See you next time!')
            break
        else:
            demo_sent = list(demo_sent.strip())
            demo_data = [(demo_sent, ['O'] * len(demo_sent))]
            tag = model.demo_one(sess, demo_data)
            PER, LOC, ORG = get_entity(tag, demo_sent)
            print('PER: {}\nLOC: {}\nORG: {}'.format(PER, LOC, ORG))



.\data_path_save\1526890202\checkpoints/model-31680


ValueError: Variable bi-lstm/bidirectional_rnn/fw/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "C:\Users\Ian\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)
